In [1]:

import pandas as pd
import os
import json
import numpy as np

def extract_mean_value(value_str):
    if "N/A" in value_str:
        return float(np.inf)
    if isinstance(value_str, str) and '±' in value_str:
        return float(value_str.split('±')[0].strip())
    elif isinstance(value_str, (int, float)):
        return float(value_str)
    else:
        raise ValueError(f"Cannot parse value: {value_str}")

def filter_min_rows_by_method(df):
    # print(df)
    target_columns = [col for col in df.columns if "Harm" in str(col)]
    
    # Initialize list to store the best rows
    selected_rows = []
    
    # Group by method
    for method, group in df.groupby('method'):
        group_copy = group.copy()
        
        numeric_values = pd.DataFrame(index=group_copy.index)
        for col in target_columns:
            if col not in df.columns: 
                continue
            numeric_values[col] = group_copy[col].apply(extract_mean_value)
        
        rows_to_keep = set()
        
        for col in target_columns:
            if col not in df.columns: 
                continue
            min_value = numeric_values[col].min()
            min_rows = numeric_values[numeric_values[col] == min_value].index
            rows_to_keep.update(min_rows)
        
        for idx in rows_to_keep:
            selected_rows.append(group.loc[idx])
    
    # Combine all selected rows into a new dataframe
    result_df = pd.DataFrame(selected_rows).reset_index(drop=True)
    
    return result_df


def collect_data_p(p: str, datasets=[], result_dir='results', train_test_flag="test"):
    
    if not datasets:
        datasets = [d for d in os.listdir(result_dir) if os.path.isdir(os.path.join(result_dir, d))]
    
    data_rows = []
    
    all_method_imputation_pairs = []
    
    for dataset in datasets:
        dataset_dir = os.path.join(result_dir, dataset)
        if not os.path.exists(dataset_dir):
            print(f"Warning: Dataset directory {dataset_dir} not found. Skipping.")
            continue
            
        # Get all method directories for this dataset
        method_dirs = [d for d in os.listdir(dataset_dir) if os.path.isdir(os.path.join(dataset_dir, d))]
        # print("dataset:", dataset_dir, "methods:", method_dirs)
        method_dirs = [e for e in method_dirs if 'Balu' not in e]

        for method in method_dirs:
            if method in ['dr']:                
                continue
            method_dir = os.path.join(dataset_dir, method)
            # print(method_dir)
            # print(os.listdir(method_dir))
            for imputation in os.listdir(method_dir):
                if ".DS" in imputation:
                    continue
                if p == '0.0' and imputation != 'full':                 # if the missing percentage is 0.0, no evaluation need for other imputation methods
                    continue
                if p != '0.0' and imputation == 'full':                 # if has missing data, there is no evaluation for complete data (full)
                    continue
                
                method_imputation_pair = (method, imputation)
                if method_imputation_pair not in all_method_imputation_pairs:
                    all_method_imputation_pairs.append(method_imputation_pair)

    # Second pass: collect results for each method/imputation pair across all datasets
    for method, imputation in all_method_imputation_pairs:
        row = {
            'method': method,
            'imputation': imputation
        }
        
        for dataset in datasets:
            dataset_dir = os.path.join(result_dir, dataset)
            if not os.path.exists(dataset_dir):
                print(f"{method_dir} not exits!")
                row[f"{dataset}_PEHE"] = "N/A"
                row[f"{dataset}_MAE"] = "N/A"
                continue
                
            method_dir = os.path.join(dataset_dir, method)
            if not os.path.exists(method_dir):
                row[f"{dataset}_PEHE"] = "N/A"
                row[f"{dataset}_MAE"] = "N/A"
                continue
            
            imputation_dir = os.path.join(method_dir, imputation)
            
            # print(imputation_dir)
            # if 'SPNet' in method_dir and imputation == 'no':
            #     print([f for f in os.listdir(imputation_dir) if f.endswith(f"pt_{train_test_flag}_results.json")])
            
            results_files = [f for f in os.listdir(imputation_dir) if f.startswith(f"p={p}") and f.endswith(f"pt_{train_test_flag}_results.json")]

            # print(results_files)
            effect_pehe_values = []
            effect_mae_values = []
            effect_sum_values = []
            
            # Collect metrics from all matching files
            for file in results_files:
                file_path = os.path.join(method_dir, imputation, file)

                try:
                    with open(file_path, 'r') as f: 
                        results = json.load(f) 
                        if results['effect_pehe'] == np.nan or results['effect_pehe'] > 50: continue
                        # if results['effect_pehe'] == np.nan or results['effect_pehe'] > 50: print(method_dir, results['effect_pehe'])
                        # if results['effect_mae'] == np.nan or results['effect_mae'] > 50: print(method_dir, results['effect_mae'])

                        if 'effect_pehe' in results:
                            effect_pehe_values.append(results['effect_pehe'])
                        if 'effect_mae' in results:
                            effect_mae_values.append(results['effect_mae'])
                        effect_sum_values.append(results['effect_pehe'] + results['effect_mae'])

                        # if method_dir == 'SPNet' and imputation == "no":
                        #     print("SPNet", results['effect_pehe'], results['effect_mae'])
                except (json.JSONDecodeError, FileNotFoundError) as e:
                    print(f"Warning: Error reading file {file_path}: {e}")
            
            # if 'SPNet' in method_dir and imputation == 'no' :
            #     print(dataset, "PEHE:", effect_pehe_values, '\nMAE:', effect_mae_values)
            # print("PHHE", effect_pehe_values)
            # print("MAE", effect_mae_values)
            # Format mean and std for each metric
            if effect_pehe_values:
                mean_pehe = np.mean(effect_pehe_values)
                std_pehe = np.std(effect_pehe_values)
                row[f"{dataset}_PEHE"] = f"{mean_pehe:.2f} ± {std_pehe:.2f}"
            else:
                row[f"{dataset}_PEHE"] = "N/A"
            
            if effect_mae_values:
                mean_mae = np.mean(effect_mae_values)
                std_mae = np.std(effect_mae_values)
                row[f"{dataset}_MAE"] = f"{mean_mae:.2f} ± {std_mae:.2f}"
            else:
                row[f"{dataset}_MAE"] = "N/A"

            sum_metric_values = [m1+m2 for m1, m2 in zip(effect_pehe_values, effect_mae_values)]
            if sum_metric_values:
                mean_m = np.mean(sum_metric_values)
                std_m = np.std(sum_metric_values)
                row[f"{dataset}_Harm"] = f"{mean_m:.2f} ± {std_m:.2f}"
            else:
                row[f"{dataset}_Harm"] = 'N/A'

        # print(row)
        data_rows.append(row)
    
    # Create DataFrame
    df = pd.DataFrame(data_rows)
    df = df.loc[(df['method'] != 'dml')]
    
    # Sort the DataFrame by method and imputation
    if not df.empty:
        df = df.sort_values(by=['method', 'imputation'])
    
    return df

def display_results(result_dir,p, datasets=["Syn", "Youtube"], train_test_flag='test'):
    import pandas as pd

    pd.set_option('display.max_rows', None)         # Show all rows
    pd.set_option('display.max_columns', None)      # Show all columns
    pd.set_option('display.width', None)            # Use the full width of the notebook cell
    pd.set_option('display.max_colwidth', None)     # Show full content of each column

    # Example: Display the DataFrame created by collect_data_p
    df = collect_data_p(p, datasets=datasets, result_dir=result_dir, train_test_flag=train_test_flag) #, "AMZS", "Flickr"])

    # dataset_map = {'Syn_M=None_SimRel=1_Rel=4':"Instagram",  'BlogCatalog1_M=20_SimRel=1_Rel=4':"BlogCatalog", 'Flickr1_M=20_SimRel=1_Rel=4':"Flickr", 'Youtube_M=20_SimRel=1_Rel=4':"Youtube"}
    dataset_map = {'Syn_M=None_SimRel=1_Rel=4':"Instagram",  'BlogCatalog1_M=20_SimRel=0_Rel=1':"BlogCatalog", 'BlogCatalog1_M=20_SimRel=1_Rel=4':"BlogCatalog", 'Flickr1_M=20_SimRel=0_Rel=1':"Flickr", 'Flickr1_M=20_SimRel=1_Rel=4':"Flickr", 'Youtube_M=20_SimRel=1_Rel=4':"Youtube"}
    
    def replace_col(col):
        for k, v in dataset_map.items():
            if k in col:
                return v + col[col.rfind("_"):]
                # return col.replace(k, v)
        return col

    def prepare_df_for_multirow(df, merge_columns):
        df_copy = df.copy()
        
        for col in merge_columns:
            # Replace consecutive duplicate values with empty string
            mask = df_copy[col] == df_copy[col].shift(1)
            df_copy.loc[mask, col] = ''
        
        return df_copy

    new_column_names = [replace_col(col) for col in df.columns]
    df.columns = new_column_names
    display(df)

    # print(df.columns)
    
    df_no = df[(df['imputation'] == 'no')]
    cols = [col for col in df_no.columns if "Harm" not in col]
    df_no['method'] = df_no['method'].apply(lambda x: "GCN-HSIC" if "GCN" in str(x) else "SAGE-HSIC" if "GraphSAGE" in str(x) else x)
    df_no['method'] = df_no['method'].apply(lambda x: "X-Learner" if str(x)=='xl' else "CausalForest" if str(x)=='cf' else "T-Learner" if 'tl' == str(x) else "R-Learner" if 'rl' == str(x) else x)

    

    if len(df_no) >= 1 and p=='0.1':
        display(df_no[cols])
        # print(df_no[cols].to_latex(index=False, escape=False))

    def mean_val(x):
        x = str(x)
        i = x.index(" ±")
        return float(x[:i])
    if p == '0.1':
        for col in df_no.columns:
            if "MAE" in col or "PEHE" in col:
                df_no[col] = df_no[col].apply(lambda x: mean_val(x))
        display(df_no[cols])

    # datasets = ['Instagram', 'BlogCatalog', 'Flickr', 'Youtube']
    # for i in range(len(target_columns)//2):
    #     target_columns[[i*2,i*2+1]]

    first_two_cols = df.iloc[:, :2]
    remaining_cols = df.iloc[:, 2:]
    j = 0
    for i in range(0, remaining_cols.shape[1], 3):
        pair_cols = remaining_cols.iloc[:, i:i+3]
        sub_df = pd.concat([first_two_cols, pair_cols], axis=1)

        sub_df = filter_min_rows_by_method(sub_df)          # filter out rows without at least one minimum value

        sub_df['method'] = sub_df['method'].apply(lambda x: "GCN-HSIC" if "GCN" in str(x) else "SAGE-HSIC" if "GraphSAGE" in str(x) else x)
        sub_df['method'] = sub_df['method'].apply(lambda x: "X-Learner" if str(x)=='xl' else "CausalForest" if str(x)=='cf' else "T-Learner" if 'tl' == str(x) else "R-Learner" if 'rl' == str(x) else x)

        cols = [col for col in sub_df.columns if "Harm" not in col]
        # print(sub_df[cols].to_latex(index=False, escape=False))


        display(sub_df[cols])
        if p == '0.1':
            df_no.reset_index(drop=True, inplace=True)
            for col in sub_df.columns:
                if "MAE" in col or "PEHE" in col:
                    sub_df[col] = sub_df[col].apply(lambda x: mean_val(x))
                    sub_df[col] = ((df_no[col] - sub_df[col]) / df_no[col])*100
            
            display(sub_df[cols])
        

    # print(df.columns)
    
    # display(df)
    # df['p'] = p
    # cols = df.columns.tolist()
    # cols = ['p'] + [col for col in cols if col != 'p']
    # df = df[cols]

    # df = prepare_df_for_multirow(df, ['p', 'method'])


    
    # latex_output = df.to_latex(index=False, escape=False)
    # print(latex_output)

# datasets = ["Syn_M=None"] #, "AMZS_M=20", "BlogCatalog1_M=20", "Flickr_M=20", "Flickr1_M=20"]
# display_results(result_dir='results_Q1',p='0.1', datasets=datasets, train_test_flag='test')

# Test with complete attributes

## MCAR

In [3]:
import warnings
warnings.filterwarnings('ignore')

datasets_all = ['Syn_M=None_SimRel=1_Rel=4',  'BlogCatalog1_M=20_SimRel=1_Rel=4', 'Flickr1_M=20_SimRel=1_Rel=4', 'Youtube_M=20_SimRel=1_Rel=4']
datasets_all = ['Syn_M=None_SimRel=1_Rel=4', 'Youtube_M=20_SimRel=1_Rel=4', 'BlogCatalog1_M=20_SimRel=0_Rel=1', 'Flickr1_M=20_SimRel=0_Rel=1']     # , 'BlogCatalog1_M=20_SimRel=1_Rel=4', 'Flickr1_M=20_SimRel=1_Rel=4'

datasets = datasets_all
datasets = [e+"_MCAR" for e in datasets]

result_dir = 'results_Q1_MAR_MCAR'
# print("#################################################################################"*3)
display_results(result_dir=result_dir,p='0.0', datasets=datasets, train_test_flag='test')
print("#################################################################################"*3)
display_results(result_dir=result_dir,p='0.1', datasets=datasets, train_test_flag='test')
print("#################################################################################"*3)
display_results(result_dir=result_dir,p='0.3', datasets=datasets, train_test_flag='test')


,method,imputation,Instagram_PEHE,Instagram_MAE,Instagram_Harm,Youtube_PEHE,Youtube_MAE,Youtube_Harm,BlogCatalog_PEHE,BlogCatalog_MAE,BlogCatalog_Harm,Flickr_PEHE,Flickr_MAE,Flickr_Harm
1,GCN_no_drop=0.1_HSIC,full,5.74 ± 1.17,1.93 ± 1.43,7.67 ± 2.35,5.88 ± 1.16,1.06 ± 0.76,6.94 ± 1.68,10.63 ± 5.02,5.65 ± 4.12,16.28 ± 9.08,12.30 ± 3.94,5.83 ± 3.25,18.13 ± 7.13
5,GraphSAGE_no_drop=0.1_HSIC,full,5.95 ± 1.35,2.45 ± 1.47,8.41 ± 2.60,5.98 ± 1.16,1.19 ± 0.98,7.17 ± 1.73,14.36 ± 7.13,4.88 ± 2.54,19.24 ± 7.71,17.97 ± 5.48,5.10 ± 4.08,23.07 ± 8.93
6,NetDeconf,full,8.15 ± 7.50,4.38 ± 8.04,12.53 ± 15.51,4.72 ± 0.62,1.23 ± 0.90,5.96 ± 1.36,7.77 ± 4.13,4.50 ± 3.25,12.26 ± 7.29,10.40 ± 2.69,5.57 ± 2.84,15.97 ± 5.30
0,SPNet,full,5.22 ± 1.00,2.36 ± 1.56,7.58 ± 2.54,4.65 ± 0.55,0.95 ± 0.67,5.60 ± 1.12,7.74 ± 3.36,4.87 ± 3.79,12.61 ± 7.08,9.43 ± 3.81,4.88 ± 3.71,14.32 ± 7.37
3,cf,full,3.87 ± 0.42,3.09 ± 0.35,6.96 ± 0.76,4.11 ± 0.51,3.29 ± 0.42,7.40 ± 0.93,8.03 ± 2.41,5.73 ± 1.99,13.76 ± 4.39,14.54 ± 1.95,8.62 ± 2.88,23.15 ± 4.78
7,rl,full,2.60 ± 0.22,2.07 ± 0.18,4.66 ± 0.39,2.58 ± 0.41,2.05 ± 0.33,4.62 ± 0.75,8.34 ± 2.42,5.85 ± 1.88,14.19 ± 4.29,15.24 ± 2.04,8.86 ± 2.72,24.09 ± 4.70
4,tl,full,3.04 ± 0.44,2.42 ± 0.36,5.46 ± 0.81,3.23 ± 0.42,2.56 ± 0.34,5.79 ± 0.76,8.31 ± 2.40,5.91 ± 1.85,14.23 ± 4.24,15.20 ± 2.05,9.01 ± 2.74,24.21 ± 4.75
8,xl,full,3.11 ± 0.36,2.48 ± 0.29,5.59 ± 0.65,3.12 ± 0.42,2.48 ± 0.34,5.61 ± 0.76,8.12 ± 2.44,5.75 ± 1.89,13.87 ± 4.32,14.94 ± 2.05,8.82 ± 2.81,23.76 ± 4.82


,method,imputation,Instagram_PEHE,Instagram_MAE
0,GCN-HSIC,full,5.74 ± 1.17,1.93 ± 1.43
1,SAGE-HSIC,full,5.95 ± 1.35,2.45 ± 1.47
2,NetDeconf,full,8.15 ± 7.50,4.38 ± 8.04
3,SPNet,full,5.22 ± 1.00,2.36 ± 1.56
4,CausalForest,full,3.87 ± 0.42,3.09 ± 0.35
5,R-Learner,full,2.60 ± 0.22,2.07 ± 0.18
6,T-Learner,full,3.04 ± 0.44,2.42 ± 0.36
7,X-Learner,full,3.11 ± 0.36,2.48 ± 0.29


,method,imputation,Youtube_PEHE,Youtube_MAE
0,GCN-HSIC,full,5.88 ± 1.16,1.06 ± 0.76
1,SAGE-HSIC,full,5.98 ± 1.16,1.19 ± 0.98
2,NetDeconf,full,4.72 ± 0.62,1.23 ± 0.90
3,SPNet,full,4.65 ± 0.55,0.95 ± 0.67
4,CausalForest,full,4.11 ± 0.51,3.29 ± 0.42
5,R-Learner,full,2.58 ± 0.41,2.05 ± 0.33
6,T-Learner,full,3.23 ± 0.42,2.56 ± 0.34
7,X-Learner,full,3.12 ± 0.42,2.48 ± 0.34


,method,imputation,BlogCatalog_PEHE,BlogCatalog_MAE
0,GCN-HSIC,full,10.63 ± 5.02,5.65 ± 4.12
1,SAGE-HSIC,full,14.36 ± 7.13,4.88 ± 2.54
2,NetDeconf,full,7.77 ± 4.13,4.50 ± 3.25
3,SPNet,full,7.74 ± 3.36,4.87 ± 3.79
4,CausalForest,full,8.03 ± 2.41,5.73 ± 1.99
5,R-Learner,full,8.34 ± 2.42,5.85 ± 1.88
6,T-Learner,full,8.31 ± 2.40,5.91 ± 1.85
7,X-Learner,full,8.12 ± 2.44,5.75 ± 1.89


,method,imputation,Flickr_PEHE,Flickr_MAE
0,GCN-HSIC,full,12.30 ± 3.94,5.83 ± 3.25
1,SAGE-HSIC,full,17.97 ± 5.48,5.10 ± 4.08
2,NetDeconf,full,10.40 ± 2.69,5.57 ± 2.84
3,SPNet,full,9.43 ± 3.81,4.88 ± 3.71
4,CausalForest,full,14.54 ± 1.95,8.62 ± 2.88
5,R-Learner,full,15.24 ± 2.04,8.86 ± 2.72
6,T-Learner,full,15.20 ± 2.05,9.01 ± 2.74
7,X-Learner,full,14.94 ± 2.05,8.82 ± 2.81


...

,method,imputation,Instagram_PEHE,Instagram_MAE,Instagram_Harm,Youtube_PEHE,Youtube_MAE,Youtube_Harm,BlogCatalog_PEHE,BlogCatalog_MAE,BlogCatalog_Harm,Flickr_PEHE,Flickr_MAE,Flickr_Harm
13,GCN_no_drop=0.1_HSIC,gain,4.35 ± 0.76,0.50 ± 0.34,4.85 ± 0.77,4.86 ± 0.42,0.71 ± 0.27,5.57 ± 0.45,8.40 ± 2.49,4.81 ± 1.59,13.21 ± 3.94,13.40 ± 2.28,4.56 ± 1.31,17.96 ± 3.39
10,GCN_no_drop=0.1_HSIC,grape_real,5.51 ± 1.02,2.53 ± 1.50,8.04 ± 2.36,6.19 ± 0.77,0.62 ± 0.60,6.81 ± 0.87,14.17 ± 14.00,4.48 ± 6.40,18.65 ± 18.52,12.91 ± 3.62,1.43 ± 1.92,14.34 ± 4.42
15,GCN_no_drop=0.1_HSIC,grape_rel=0.0,5.77 ± 1.71,2.56 ± 2.00,8.33 ± 3.58,6.77 ± 0.95,0.81 ± 0.57,7.59 ± 1.14,8.78 ± 3.77,2.19 ± 1.73,10.97 ± 4.31,10.44 ± 2.23,1.27 ± 0.88,11.71 ± 2.80
8,GCN_no_drop=0.1_HSIC,knn,5.37 ± 1.05,2.02 ± 0.96,7.40 ± 1.29,5.77 ± 0.51,0.54 ± 0.44,6.31 ± 0.75,6.60 ± 2.26,1.71 ± 1.70,8.32 ± 3.70,11.34 ± 2.20,2.39 ± 1.67,13.73 ± 3.69
9,GCN_no_drop=0.1_HSIC,mean,5.67 ± 0.93,1.68 ± 0.87,7.34 ± 1.71,5.89 ± 0.51,0.91 ± 0.68,6.80 ± 0.98,7.02 ± 2.66,2.51 ± 1.62,9.53 ± 4.14,8.84 ± 1.16,1.09 ± 0.72,9.93 ± 1.56
14,GCN_no_drop=0.1_HSIC,mice,7.88 ± 1.67,2.06 ± 1.81,9.94 ± 3.29,7.53 ± 0.89,1.02 ± 0.60,8.55 ± 1.27,8.39 ± 2.93,4.72 ± 2.38,13.11 ± 5.09,9.88 ± 1.75,2.14 ± 1.00,12.02 ± 2.56
12,GCN_no_drop=0.1_HSIC,missforest,5.85 ± 1.31,2.35 ± 2.27,8.20 ± 3.49,6.33 ± 0.65,1.00 ± 0.61,7.33 ± 0.81,9.49 ± 3.03,5.65 ± 3.52,15.14 ± 6.23,14.09 ± 2.84,5.37 ± 3.32,19.45 ± 5.75
11,GCN_no_drop=0.1_HSIC,no,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A
45,GraphSAGE_no_drop=0.1_HSIC,gain,4.18 ± 0.81,0.89 ± 0.99,5.07 ± 1.79,5.27 ± 0.46,1.08 ± 0.72,6.35 ± 1.04,11.40 ± 4.49,4.25 ± 2.27,15.65 ± 5.25,17.77 ± 6.05,3.67 ± 1.99,21.44 ± 6.83
42,GraphSAGE_no_drop=0.1_HSIC,grape_real,6.50 ± 1.97,4.02 ± 2.56,10.52 ± 4.47,6.02 ± 0.73,0.84 ± 0.51,6.85 ± 1.00,15.52 ± 8.99,7.11 ± 3.86,22.63 ± 11.09,16.36 ± 3.00,3.13 ± 1.77,19.49 ± 2.10


,method,imputation,Instagram_PEHE,Instagram_MAE
0,GCN-HSIC,gain,4.35 ± 0.76,0.50 ± 0.34
1,SAGE-HSIC,gain,4.18 ± 0.81,0.89 ± 0.99
2,NetDeconf,gain,4.23 ± 0.63,0.72 ± 0.44
3,SPNet,gain,4.05 ± 0.64,1.05 ± 1.10
4,CausalForest,gain,3.58 ± 0.35,2.85 ± 0.30
5,R-Learner,gain,3.12 ± 0.13,2.48 ± 0.12
6,T-Learner,gain,3.16 ± 0.29,2.51 ± 0.24
7,X-Learner,gain,3.13 ± 0.28,2.48 ± 0.23


,method,imputation,Youtube_PEHE,Youtube_MAE
0,GCN-HSIC,gain,4.86 ± 0.42,0.71 ± 0.27
1,SAGE-HSIC,gain,5.27 ± 0.46,1.08 ± 0.72
2,NetDeconf,grape_rel=0.0,4.72 ± 0.63,0.64 ± 0.29
3,SPNet,grape_rel=0.0,4.81 ± 0.64,0.45 ± 0.28
4,CausalForest,gain,4.03 ± 0.30,3.22 ± 0.25
5,R-Learner,gain,3.41 ± 0.27,2.71 ± 0.20
6,T-Learner,gain,3.60 ± 0.27,2.87 ± 0.23
7,X-Learner,gain,3.52 ± 0.29,2.82 ± 0.23


,method,imputation,BlogCatalog_PEHE,BlogCatalog_MAE
0,GCN-HSIC,knn,6.60 ± 2.26,1.71 ± 1.70
1,SAGE-HSIC,gain,11.40 ± 4.49,4.25 ± 2.27
2,NetDeconf,knn,6.90 ± 2.45,2.73 ± 2.02
3,SPNet,mean,6.52 ± 1.39,2.62 ± 1.12
4,CausalForest,mice,8.20 ± 2.40,5.63 ± 1.75
5,R-Learner,mice,8.31 ± 2.37,5.65 ± 1.64
6,T-Learner,mice,8.36 ± 2.43,5.69 ± 1.65
7,X-Learner,mice,8.29 ± 2.45,5.63 ± 1.67


,method,imputation,Flickr_PEHE,Flickr_MAE
0,GCN-HSIC,mean,8.84 ± 1.16,1.09 ± 0.72
1,SAGE-HSIC,grape_real,16.36 ± 3.00,3.13 ± 1.77
2,NetDeconf,grape_rel=0.0,9.70 ± 2.82,2.73 ± 1.60
3,SPNet,mice,9.80 ± 1.98,2.96 ± 2.62
4,CausalForest,mean,15.14 ± 2.06,7.09 ± 1.59
5,R-Learner,mice,15.11 ± 2.28,7.89 ± 2.58
6,T-Learner,mice,15.30 ± 2.40,8.09 ± 2.48
7,X-Learner,knn,15.58 ± 2.30,7.79 ± 2.00
